In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from venn import venn




In [ ]:
#Import eggnog files and build a mapping

#Start with K. marxianus to S. cerevisiae and inverse.  

#Get gene expression for each species

#Merge gene expression data across species

#Scatter plot across species. 

In [22]:
base_dir = os.path.normpath('C:/Users/heineib/Google Drive/Crick_LMS/projects/9_strains')


In [23]:
orthtable = pd.read_table( base_dir + os.sep + os.path.normpath('eggnog_mapper/Kmar/genemark/out.emapper.orthologs'), skiprows = 4, skipfooter=3, engine='python')

In [24]:
orthtable_scer = orthtable[orthtable['species']=='Saccharomyces cerevisiae(4932)']

In [74]:
#orth_types = ['one2one', 'many2one', 'one2many', 'many2many']

source = 'Kmar'
target = 'Scer'

orthgroups = orthtable_scer.groupby('orth_type')

#for orth_type in orth_types:



data = []

#Append data with source_genename, target_genename, orth_type

for orth_type in ['one2one', 'many2one', 'one2many', 'many2many']: 
    orthtable_scer_type = orthgroups.get_group(orth_type)

    for row in orthtable_scer_type.iterrows():
        source_genename = row[1]['#query']
        row_orths_raw = row[1]['orthologs']
        row_orths = row_orths_raw.split(',')
        if orth_type in {'one2many', 'many2many'}:
            if len(row_orths)==1: 
                print("{} should have more than one ortholog, but only has one: {}".format(source_genename,row_orths_raw )) 
        for row_orth in row_orths: 
            if row_orth[0]=='*': 
                data.append((source_genename, row_orth.split('*')[1],orth_type))
            else: 
                data.append((source_genename, row_orth[0],orth_type))


NC_036025.1:1089523-1090612(-)_1 should have more than one ortholog, but only has one: *YJR153W


In [71]:
    orthtable_scer_type = orthgroups.get_group(orth_type)

    for row in orthtable_scer_type.iterrows():
        row_orth = row[1]['orthologs']
        assert row_orth[0]=='*', 'No * at the beginning of ortholog'
        data.append((row[1]['#query'], row[1]['orthologs'].split('*')[1],orth_type))

NC_036025.1:1089523-1090612(-)_1 should have more than one ortholog, but only has one: *YJR153W


In [ ]:
#Said it was one to many but only had one ortholog: NC_036025.1:1089523-1090612(-)_1	one2many	Saccharomyces cerevisiae(4932)	*YJR153W

In [76]:
orth_map = pd.DataFrame(data, columns=[source + '_genename', target+'_genename', 'orth_type'])

In [80]:
orth_map.to_csv(base_dir + os.sep + os.path.normpath('eggnog_mapper/')+ source + '_' + target + '.csv')